#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [2]:
for doc in docs:
    print(f'{doc}\n==================END===================')

page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.

Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.

â€¦' metadata={'source': 'speech.txt'}
==================END===================
page_content='â€¦

It will be all the easier for us to conduct ourselves as belligere

In [3]:
embeddings=OllamaEmbeddings(model="nomic-embed-text")
db=FAISS.from_documents(docs,embeddings)
db

C:\Users\Utkar\AppData\Local\Temp\ipykernel_57912\1468083222.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model="nomic-embed-text")


In [4]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [5]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [6]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='79bb648a-710c-406f-b06b-5b40fee88928', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(343.23434)),
 (Document(id='a6449797-b0a1-48f4-b52e-81675c77dfea', metadata={'source': 's

In [7]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-0.3020259737968445,
 0.27965277433395386,
 -3.6142232418060303,
 -0.9986158609390259,
 2.0197677612304688,
 1.4683958292007446,
 -1.0225001573562622,
 0.2344488948583603,
 0.4826875329017639,
 -0.18012651801109314,
 -0.0629497766494751,
 0.5970190167427063,
 0.8667795658111572,
 1.6235206127166748,
 2.0871212482452393,
 -0.781399667263031,
 0.2582467794418335,
 -1.2086212635040283,
 -0.6825215816497803,
 0.7261427640914917,
 -0.9009966850280762,
 -0.39372825622558594,
 0.09221303462982178,
 0.479297012090683,
 1.344508171081543,
 0.41011476516723633,
 -0.5244297981262207,
 0.671367883682251,
 -1.0714606046676636,
 -0.5256907939910889,
 0.9152607917785645,
 0.06177143380045891,
 -0.05081813782453537,
 0.28206780552864075,
 -1.5362272262573242,
 -1.917377233505249,
 0.07924095541238785,
 1.3351359367370605,
 0.35592061281204224,
 -1.428046703338623,
 0.062106337398290634,
 -0.4961971938610077,
 0.12286541610956192,
 -1.5529093742370605,
 1.3663420677185059,
 -0.689302921295166,
 -0.130

In [8]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='79bb648a-710c-406f-b06b-5b40fee88928', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='a6449797-b0a1-48f4-b52e-81675c77dfea', metadata={'source': 'speech.txt'}, page_content='â

In [9]:
### Saving And Loading
db.save_local("faiss_index")

In [10]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [11]:
docs

[Document(id='79bb648a-710c-406f-b06b-5b40fee88928', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='a6449797-b0a1-48f4-b52e-81675c77dfea', metadata={'source': 'speech.txt'}, page_content='â